In [6]:
files='./NLP/NER/data/BosonNLP_NER_6C.txt'
a={'I-TIME', 'I-PER', 'B-CRIME', 'I-CRIME', 'B-TIME', 'B-LOC', 'B-ORG', 
   'B-ROLE', 'B-PER', 'I-ROLE', 'I-ORG', 'I-LOC', 'I-LAW', 'B-LAW', 'O'}
import jieba,re
f=open(files,'r')
lines=f.readlines()
compiles=re.compile("{{(.*?)}}")
filter_compile=re.compile("[a-zA-Z_]+:")
compile_url=re.compile("(http[s]?:[a-zA-Z0-9\.\/\?#&\+=]+)")
tags=[]
words=[]
for line in lines:
    split_line=compiles.split(line)
    tag=[]
    word=[]
    for lin in split_line:
        if len(lin.strip())==0:
            continue
        lin=compile_url.sub("URL",lin)
        if filter_compile.match(lin):
            entity_name,entity=lin.split(":",1)
            entity=list(jieba.cut(entity))
            entities=[a.strip() for a in entity if len(a.strip())!=0]
            word.extend(entities)
            if entity_name=='person_name':
                tag.append("B-PER")
                if len(entity)!=0:
                    tag.extend(["I-PER"]*(len(entities)-1))
            elif entity_name=='time':
                tag.append("B-TIME")
                if len(entity)!=0:
                    tag.extend(["I-TIME"]*(len(entities)-1))
            elif entity_name=='location':
                tag.append("B-LOC")
                if len(entity)!=0:
                    tag.extend(["I-LOC"]*(len(entities)-1))
            elif entity_name=='org_name':
                tag.append("B-ORG")
                if len(entity)!=0:
                    tag.extend(["I-ORG"]*(len(entities)-1))
            elif entity_name=='company_name':
                tag.append("B-COMP")
                if len(entity)!=0:
                    tag.extend(["I-COMP"]*(len(entities)-1))
            elif entity_name=='product_name':
                tag.append("B-PRO")
                if len(entity)!=0:
                    tag.extend(["I-PRO"]*(len(entities)-1))
        else:
            entity=list(jieba.cut(lin))
            entities=[a.strip() for a in entity if len(a.strip())!=0]
            word.extend(entities)
            tag.extend(["O"]*len(entities))
    if len(word)!=len(tag):
        raise ValueError(len(word),len(tag),line)
    words.append(word)
    tags.append(tag)
print(len(words),len(tags))
files1='./NLP/NER/data/target.txt'
files2='./NLP/NER/data/source.txt'
f1=open(files1,'w')
for tag in tags:
    f1.write(" ".join(tag)+"\n")
f1.close()
f2=open(files2,'w')
for word in words:
    f2.write(" ".join(word)+"\n")
f2.close()

2000 2000


In [2]:
line='{{location:韩国}} {{product_name:doctorcos氨基酸核糖睡眠面膜}}，一盒足以满足改善细纹，美白，收细毛孔，修护肌肤，改善敏感，补水保湿，紧致肌肤！涂到脸上抹匀后按摩就会有一颗颗的小水珠出来！所以称作爆水神器，补水效果很明显，跟面霜一样涂上一层按摩一下就睡觉，{{time:第二天}}脸摸起来超级嫩滑饱满，也非常好上妆！'

entity_tag={"TIME":"time","LOC":"location","PER":"person_name","ORG":"org_name","COMP":"company_name","PRO":"product_name"}
import jieba,re
def drop_tag(seqence,tag):
    compiles=re.compile("{{(.*?)}}")
    filter_compile=re.compile("[a-zA-Z_]+:")
    seqences=compiles.split(seqence)
    seqs=[]
    for seq in seqences:
        if len(seq.strip())==0:
            continue
        if filter_compile.match(seq):
            name,entity=seq.split(":",1)
            if name==entity_tag[tag]:
                seqs.append(" START "+entity+" END ")
            else:
                seqs.append(entity)
        else:
            seqs.append(seq)
    return "".join(seqs)

def cut_seq(seqence):
    compile_url=re.compile("(http[s]?:[a-zA-Z0-9\.\/\?#&\+=]+)")
    seqence=compile_url.sub("URL",seqence)
    cuts=list(jieba.cut(seqence,HMM=False))
    cut=[a.strip() for a in cuts if len(a.strip())!=0]
    return cut

def find_tag(seqs,tag="START"):
    if isinstance(tag,list):
        index=[ind for ind,word in enumerate(seqs) if word in tag]
    else:
        index=[ind for ind,word in enumerate(seqs) if word==tag]
    return index

def locEntity(start,end,seqs,tag):
    xrange=end-start-1
    piece_tag=seqs[start+1:end]
    if xrange==1:
        tag.append('J-PER')
    elif xrange==2:
        if len(piece_tag[0])>=2:
            if piece_tag[0] in doubelsurname:
                tag.append("C-PER")
            else:
                tag.append("I-PER")
                
            if len(piece_tag[1])==2:
                tag.append("K-PER")
            else:
                tag.append("E-PER")
        else:
            tag.append("C-PER")
            if len(piece_tag[1])>=2:
                tag.append("K-PER")
            else:
                tag.append("F-PER")
    else:
        if len(piece_tag[0])>=2:
            if piece_tag[0] in doubelsurname:
                tag.append("C-PER")
            else:
                tag.append("I-PER")
        else:
            tag.append("C-PER")
        
        for i in range(1,len(piece_tag)-1):
            if len(piece_tag[i])>=2:
                tag.append("K-PER")
            else:
                tag.append("D-PER")
        if len(piece_tag[-1])>=2:
            tag.append("K-PER")
        else:
            tag.append("E-PER")
            

files='./NLP/NER/data/BosonNLP_NER_6C.txt'
lines=open(files,'r').readlines()
# for line in lines:
seqences=drop_tag(line,"LOC")
seqs=cut_seq(seqences)
init_seq=re.sub("START|END",'',seqences)
init_seqs=cut_seq(init_seq)
print(init_seqs )
new_seq=[]
tag=[]
index=find_tag(seqs,tag=["START","END"])
index=sorted(index)
reduce=[]
for i in range(0,len(index),2):
    reduce.append(index[i+1]-index[i])
entity_length=[]
j=-1
judge=True
start=0
end=0
last=0
i=0
while j<len(seqs)-1:
    j+=1
    if seqs[j]=="START":
        if len(tag)>0:
            if tag[-1]!="X-PER" and tag[-1]=="Z-PER":
                tag[-1]="A-PER"
        start=j
        j=end=reduce[i]+start
        locEntity(start,end,seqs,tag)
        i+=1
        if len(reduce)>i:
            if seqs[j+1]=="START":
                continue
            if seqs[j+2]=='START':
                tag.append("X-PER")
            else:
                tag.append('B-PER')
            j+=1
        else:
            if len(seqs)>j+1:
                tag.append('B-PER')
                j+=1
    else:
        tag.append("Z-PER")

    
print(init_seqs)
print(tag)
    
    

['韩国', 'doctorcos', '氨基酸', '核糖', '睡眠', '面膜', '，', '一盒', '足以', '满足', '改善', '细纹', '，', '美', '白', '，', '收', '细', '毛孔', '，', '修护', '肌肤', '，', '改善', '敏感', '，', '补水', '保湿', '，', '紧', '致', '肌肤', '！', '涂', '到', '脸上', '抹', '匀', '后', '按摩', '就', '会', '有', '一颗颗', '的', '小', '水珠', '出来', '！', '所以', '称作', '爆', '水', '神器', '，', '补水', '效果', '很', '明显', '，', '跟', '面霜', '一样', '涂上一层', '按摩', '一下', '就', '睡觉', '，', '第二天', '脸', '摸', '起来', '超级', '嫩', '滑', '饱满', '，', '也', '非常', '好', '上妆', '！']
['J-PER', 'B-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER', 'Z-PER'